# Tutorial 4: Bootstrap Inference

This tutorial covers **bootstrap inference** for GMM/SMM estimation - how to compute standard errors and confidence intervals using bootstrap methods.

## What You'll Learn

1. Why bootstrap inference is useful
2. How bootstrap works for moment estimators
3. Bootstrap standard errors and confidence intervals
4. Comparing asymptotic vs bootstrap inference

## Prerequisites

- Completed Tutorials 1-3
- Basic understanding of bootstrap methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from momentest import (
    smm_estimate,
    SMMEngine,
    EstimationSetup,
    bootstrap,
    table_estimates,
    table_bootstrap,
    plot_bootstrap_distribution,
    load_econ381,
)

np.random.seed(42)

## 1. Why Bootstrap?

### Asymptotic Standard Errors

The standard approach uses **asymptotic theory**:

$$\sqrt{n}(\hat{\theta} - \theta_0) \xrightarrow{d} N(0, V)$$

where $V$ is the asymptotic variance (sandwich formula).

### Problems with Asymptotic SE

1. **Finite sample bias**: Asymptotic approximation may be poor
2. **Complex models**: Sandwich formula may be hard to compute
3. **Non-standard cases**: Weak identification, boundary issues

### Bootstrap Solution

Bootstrap provides:
- **Finite sample** inference
- **No need** for analytical variance formulas
- **Robust** to model misspecification

## 2. Bootstrap for SMM

For SMM, bootstrap works by **re-drawing shocks**:

1. **Original estimation**: Use seed $s_0$ → get $\hat{\theta}$
2. **Bootstrap replication $b$**: Use seed $s_b$ → get $\hat{\theta}^{(b)}$
3. **Repeat** B times
4. **Compute**: SE = std($\hat{\theta}^{(1)}, ..., \hat{\theta}^{(B)}$)

This captures the **simulation uncertainty** in SMM.

## 3. Example: Truncated Normal

Let's use the truncated normal example from Tutorial 2:

In [ ]:
import scipy.stats as sts

# Load data
dataset = load_econ381()
data = dataset['data']
cut_lb, cut_ub = dataset['bounds']
N = len(data)

# Target moments
data_mean = data.mean()
data_var = data.var()
data_moments = [data_mean, data_var]

print(f"Data: N={N}, mean={data_mean:.2f}, var={data_var:.2f}")

In [ ]:
# Define simulation and moment functions
def trunc_norm_draws(unif_vals, mu, sigma, cut_lb, cut_ub):
    sigma = max(sigma, 1e-6)
    cut_lb_cdf = sts.norm.cdf(cut_lb, loc=mu, scale=sigma)
    cut_ub_cdf = sts.norm.cdf(cut_ub, loc=mu, scale=sigma)
    cdf_range = cut_ub_cdf - cut_lb_cdf
    if cdf_range < 1e-10:
        return np.full_like(unif_vals, (cut_lb + cut_ub) / 2)
    unif_scaled = unif_vals * cdf_range + cut_lb_cdf
    unif_scaled = np.clip(unif_scaled, cut_lb_cdf + 1e-10, cut_ub_cdf - 1e-10)
    return np.clip(sts.norm.ppf(unif_scaled, loc=mu, scale=sigma), cut_lb, cut_ub)

def sim_func(theta, shocks):
    mu, sigma = theta
    sigma = max(sigma, 1.0)
    return trunc_norm_draws(shocks, mu, sigma, cut_lb, cut_ub)

def moment_func(sim_data):
    return np.column_stack([np.mean(sim_data, axis=1), np.var(sim_data, axis=1)])

## 4. Point Estimation

In [ ]:
# Point estimation
result = smm_estimate(
    sim_func=sim_func,
    moment_func=moment_func,
    data_moments=data_moments,
    bounds=[(0, 1000), (1, 500)],
    n_sim=300,
    shock_dim=N,
    seed=42,
    weighting="optimal",
)

print(f"Point estimates: μ={result.theta[0]:.2f}, σ={result.theta[1]:.2f}")
print(f"Asymptotic SE:   SE(μ)={result.se[0]:.2f}, SE(σ)={result.se[1]:.2f}")

## 5. Bootstrap Inference

Now let's compute bootstrap standard errors and confidence intervals:

In [ ]:
# Create engine for bootstrap
engine = SMMEngine(
    k=2,
    p=2,
    n_sim=300,
    shock_dim=N,
    sim_func=sim_func,
    moment_func=moment_func,
    seed=42,
)

# Setup for bootstrap
setup = EstimationSetup(
    mode="SMM",
    model_name="truncated_normal",
    moment_type="mean_variance",
    k=2,
    p=2,
    n_sim=300,
    shock_dim=N,
    seed=42,
    weighting="optimal",
)

In [ ]:
# Run bootstrap (this takes a minute)
print("Running bootstrap with 100 replications...")
print("(This may take a minute)")

boot_result = bootstrap(
    setup=setup,
    data_moments=np.array(data_moments),
    bounds=[(0, 1000), (1, 500)],
    n_boot=100,           # Number of bootstrap replications
    alpha=0.05,           # 95% confidence intervals
    n_global=30,          # Fewer global points for speed
    n_jobs=1,             # Sequential (use -1 for parallel)
    engine=engine,
)

print(f"\nBootstrap completed: {boot_result.n_converged}/{boot_result.n_boot} converged")

In [ ]:
# Display bootstrap results
print(table_bootstrap(
    bootstrap_estimates=boot_result.bootstrap_estimates,
    theta_hat=boot_result.theta_hat,
    param_names=["μ", "σ"],
    alpha=0.05,
))

## 6. Compare Asymptotic vs Bootstrap SE

In [ ]:
print("\n" + "="*60)
print("COMPARISON: Asymptotic vs Bootstrap")
print("="*60)
print(f"{'Parameter':<10} {'Estimate':>10} {'Asymp SE':>12} {'Boot SE':>12}")
print("-"*50)
print(f"{'μ':<10} {result.theta[0]:>10.2f} {result.se[0]:>12.2f} {boot_result.se[0]:>12.2f}")
print(f"{'σ':<10} {result.theta[1]:>10.2f} {result.se[1]:>12.2f} {boot_result.se[1]:>12.2f}")
print("="*60)

## 7. Bootstrap Confidence Intervals

Bootstrap provides **percentile confidence intervals**:

$$CI_{1-\alpha} = [\hat{\theta}^{(\alpha/2)}, \hat{\theta}^{(1-\alpha/2)}]$$

where $\hat{\theta}^{(q)}$ is the $q$-th percentile of bootstrap estimates.

In [ ]:
print("\n95% Confidence Intervals:")
print(f"{'Parameter':<10} {'Lower':>12} {'Upper':>12}")
print("-"*35)
print(f"{'μ':<10} {boot_result.ci_lower[0]:>12.2f} {boot_result.ci_upper[0]:>12.2f}")
print(f"{'σ':<10} {boot_result.ci_lower[1]:>12.2f} {boot_result.ci_upper[1]:>12.2f}")

## 8. Visualize Bootstrap Distribution

In [ ]:
# Plot bootstrap distributions
fig = plot_bootstrap_distribution(
    bootstrap_estimates=boot_result.bootstrap_estimates,
    theta_hat=boot_result.theta_hat,
    param_names=["μ", "σ"],
    ci_alpha=0.05,
)
plt.show()

## 9. Bootstrap Bias

Bootstrap also reveals **bias**:

$$\text{Bias} = E[\hat{\theta}^*] - \hat{\theta}$$

where $E[\hat{\theta}^*]$ is the mean of bootstrap estimates.

In [ ]:
# Compute bootstrap bias
valid_mask = ~np.any(np.isnan(boot_result.bootstrap_estimates), axis=1)
boot_mean = np.mean(boot_result.bootstrap_estimates[valid_mask], axis=0)
bias = boot_mean - boot_result.theta_hat

print("Bootstrap Bias:")
print(f"  μ: {bias[0]:.2f} (relative: {100*bias[0]/boot_result.theta_hat[0]:.1f}%)")
print(f"  σ: {bias[1]:.2f} (relative: {100*bias[1]/boot_result.theta_hat[1]:.1f}%)")

## 10. Effect of Number of Bootstrap Replications

More replications → more precise SE estimates:

In [ ]:
# Show how SE estimate stabilizes with more replications
valid_estimates = boot_result.bootstrap_estimates[valid_mask]
n_valid = len(valid_estimates)

se_by_n = []
n_values = list(range(10, n_valid + 1, 5))

for n in n_values:
    se_n = np.std(valid_estimates[:n], axis=0, ddof=1)
    se_by_n.append(se_n)

se_by_n = np.array(se_by_n)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for i, (ax, param) in enumerate(zip(axes, ['μ', 'σ'])):
    ax.plot(n_values, se_by_n[:, i], 'b-', linewidth=2)
    ax.axhline(boot_result.se[i], color='red', linestyle='--', label='Final SE')
    ax.set_xlabel('Number of Bootstrap Replications')
    ax.set_ylabel(f'SE({param})')
    ax.set_title(f'Bootstrap SE Convergence: {param}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("SE estimates stabilize as n_boot increases.")
print("Typically 200-500 replications are sufficient.")

## 11. When to Use Bootstrap vs Asymptotic SE

| Situation | Recommendation |
|-----------|----------------|
| Large sample, simple model | Asymptotic (faster) |
| Small sample | Bootstrap |
| Complex model | Bootstrap |
| Weak identification | Bootstrap |
| Publication-quality | Both (for robustness) |

## 12. Exercises

### Exercise 1: More Replications
Run bootstrap with n_boot=200 and n_boot=500. How much do SE estimates change?

### Exercise 2: Parallel Bootstrap
Use `n_jobs=-1` to run bootstrap in parallel. How much faster is it?

### Exercise 3: Coverage
Run a Monte Carlo: generate data, estimate, compute bootstrap CI. What fraction contain the true value?

In [ ]:
# Exercise 1 starter code
# (Warning: this takes several minutes)

# for n_boot in [50, 100, 200]:
#     boot_ex = bootstrap(
#         setup=setup,
#         data_moments=np.array(data_moments),
#         bounds=[(0, 1000), (1, 500)],
#         n_boot=n_boot,
#         n_global=30,
#         n_jobs=1,
#         engine=engine,
#     )
#     print(f"n_boot={n_boot}: SE(μ)={boot_ex.se[0]:.2f}, SE(σ)={boot_ex.se[1]:.2f}")

## Summary

In this tutorial, you learned:

1. **Why bootstrap**: Finite sample inference, no analytical formulas needed
2. **How it works**: Re-estimate with different random seeds
3. **Bootstrap SE**: Standard deviation of bootstrap estimates
4. **Bootstrap CI**: Percentile confidence intervals
5. **Bias detection**: Compare bootstrap mean to point estimate

### Key Takeaways

- Bootstrap is robust and easy to implement
- Use 200+ replications for stable SE estimates
- Parallel execution speeds up computation
- Compare to asymptotic SE for robustness check

### Next Steps

- **Tutorial 5**: Diagnostics and visualization
- **Tutorial 6**: Advanced structural models